In [46]:
from finetune import Classifier, config
import tensorflow as tf
import csv
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import classification_report
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.20.0.


Get Data

In [47]:
filePath = "../Datasets/combined_trainingdata_20181013.tsv"
data = pandas.read_csv(filePath,sep='\t')
print(data.shape)
print(data.loc[0])

(1278129, 10)
index                                                             0
COMPLAINT_ID                                        US_CHICAGO_1725
CITY                                                     US_CHICAGO
COMPLAINT DATE                                           10/03/2011
DEPT_311                                         health_environment
CODE_311                           permits issued by doe work order
CATEGORY_MAIN                                           environment
CATEGORY_SUB                                    environment_general
COMPLAINT_1       QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
COMPLAINT_2       [INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...
Name: 0, dtype: object


Remove rows with no complaint 1 data. Create Complaint column that has (complaint 2 or complaint 1)

In [48]:
print(data[data.COMPLAINT_1.isna()].shape)
print(data[data.COMPLAINT_2.isna()].shape)
print(data[(data.COMPLAINT_1.isna()) & (data.COMPLAINT_2.isna())].shape)
print(data[(data.COMPLAINT_1.isna())].shape)

(14025, 10)
(964291, 10)
(13579, 10)
(14025, 10)


In [51]:
print(data[(data.COMPLAINT_1.isna())].shape)
dataFiltered = data.dropna(subset = ["COMPLAINT_1"])
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna())].shape)
print(dataFiltered[(dataFiltered.COMPLAINT_1.isna()) & (dataFiltered.COMPLAINT_2.isna())].shape)

(14025, 10)
(0, 10)
(0, 10)


In [86]:
def getComplaint(row):
    if not pandas.isnull(row.get("COMPLAINT_2")):
        return row.get("COMPLAINT_2")
    return row.get("COMPLAINT_1")
results = dataFiltered.apply(lambda row: getComplaint (row),axis=1)
# dataFiltered["complaint"] = dataFiltered.apply(lambda row: getComplaint (row),axis=1)
print(results[results.isna()].shape)

(0,)


In [87]:
dataFiltered["complaint"] = results
# print(dataFiltered[(dataFiltered.COMPLAINT_1.isna())].shape)
# print(dataFiltered.loc[0])
print(dataFiltered[dataFiltered.complaint.isna()].shape)

(0, 12)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Filter the classes (TODO)

In [102]:
print(dataFiltered.CATEGORY_SUB.unique())

['environment_air_pollution' 'environment_abandoned_site'
 'publicorder_noise_complaint' 'environment_hazardous_material'
 'environment_dumping' 'environment_asbestos ' 'environment_general'
 'planning_general' 'infrastructure_general' 'environment_water_pollution'
 'environment_recycling' 'street_repair' 'housing_general'
 'environment_overgrowth' 'housing_health_code' 'street_sewar'
 'environment_garbage_collection' 'housing_safety'
 'environment_abandoned_vehicle' 'governance_signage' 'street_general'
 'environment_litter' 'street_urgent_repair' 'street_sidewalk'
 'environment_dead_animal' 'street_cleaning' 'environemnt_dead_animal'
 'vandalism_general' 'planning_construction' 'planning_unsafe_environment'
 'infrastructure_water' 'infrastructure_water_repair'
 'environnment_hazardous_material' 'vandalism_graffiti' 'street_lighting'
 'publichealth_general' 'publichealth_school_hygiene' 'fire_general'
 'street_parking' 'housing_mold' 'publichealth_pests' 'housing_pests'
 'street_slipp

Filter data only to ones with at least 10 characters and at most 512

In [88]:
mask = (dataFiltered["complaint"].str.len() >=10) & (dataFiltered["complaint"].str.len() <=512)
dataFiltered = dataFiltered.loc[mask]
print(dataFiltered.shape)

(1084601, 12)


Prepare Training Data

In [91]:
trainingData = dataFiltered[["complaint", "CATEGORY_SUB"]]
print(type(trainingData))
print(trainingData.shape)

<class 'pandas.core.frame.DataFrame'>
(1084601, 2)


Get stratified sample to test with (use 10% of the data)

In [95]:
_, sampleX, _, sampleY = train_test_split(trainingData.complaint, trainingData.CATEGORY_SUB, test_size=0.1, random_state=42, stratify=trainingData.CATEGORY_SUB)
print(sampleY.shape)

(108461,)


Split sample into train/test (80/20)

In [97]:
trainX, testX, trainY, testY = train_test_split(sampleX, sampleY, test_size=0.2, random_state=42, stratify=sampleY)
print(trainX.shape)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.